In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix

In [2]:
newsgroups_data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

In [3]:
import numpy as np

def load_images(file_path):
    with open(file_path, 'rb') as f:
        # Read the magic number and metadata
        magic_number = int.from_bytes(f.read(4), byteorder='big')  # Magic number
        num_images = int.from_bytes(f.read(4), byteorder='big')    # Number of images
        num_rows = int.from_bytes(f.read(4), byteorder='big')      # Rows per image
        num_cols = int.from_bytes(f.read(4), byteorder='big')      # Columns per image

        # Read the image data
        images = np.frombuffer(f.read(), dtype=np.uint8)
        images = images.reshape(num_images, num_rows, num_cols)    # Reshape into 3D array
    return images

def load_labels(file_path):
    with open(file_path, 'rb') as f:
        # Read the magic number and metadata
        magic_number = int.from_bytes(f.read(4), byteorder='big')  # Magic number
        num_labels = int.from_bytes(f.read(4), byteorder='big')    # Number of labels

        # Read the label data
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

# Example usage

train_images_file = 'Mnist/train-images.idx3-ubyte'
train_labels_file = 'Mnist/train-labels.idx1-ubyte'
test_images_file = 'Mnist/t10k-images.idx3-ubyte'
test_labels_file = 'Mnist/t10k-labels.idx1-ubyte'

mnist_train_images = load_images(train_images_file)
mnist_train_labels = load_labels(train_labels_file)
mnist_test_images = load_images(test_images_file)
mnist_test_labels = load_labels(test_labels_file)


print(f"Number of images: {mnist_train_images.shape[0]}")
print(f"Image shape: {mnist_test_images.shape[0:]}")  # Rows x Columns
print(f"Image shape: {mnist_train_images.shape[1:]}")  # Rows x Columns
print(f"First label: {mnist_train_labels[0]}")

Number of images: 60000
Image shape: (10000, 28, 28)
Image shape: (28, 28)
First label: 5


In [4]:
import numpy as np

train_images_file = 'Fashion_Mnist/train-images-idx3-ubyte'
train_labels_file = 'Fashion_Mnist/train-labels-idx1-ubyte'
test_images_file = 'Fashion_Mnist/t10k-images-idx3-ubyte'
test_labels_file = 'Fashion_Mnist/t10k-labels-idx1-ubyte'

fashion_mnist_train_images = load_images(train_images_file)
fashion_mnist_train_labels = load_labels(train_labels_file)
fashion_mnist_test_images = load_images(test_images_file)
fashion_mnist_test_labels = load_labels(test_labels_file)


print(f"Number of images: {fashion_mnist_train_images.shape[0]}")
print(f"Image shape: {fashion_mnist_test_images.shape[0:]}")  # Rows x Columns
print(f"Image shape: {fashion_mnist_train_images.shape[1:]}")  # Rows x Columns
print(f"First label: {fashion_mnist_train_labels[0]}")

Number of images: 60000
Image shape: (10000, 28, 28)
Image shape: (28, 28)
First label: 9


# Problem 1

### Question 1


The objective function in K-Means clustering is to minimize the within-cluster sum of squared distances:

$$
J(\pi, \mu) = \sum_{i=1}^{N} \sum_{k=1}^{K} \pi_{ik} \| x_i - \mu_k \|^2
$$

where:

- $ x_i $ is the $ i $-th data point,
- $ \mu_k $ is the centroid of cluster $ k $,
- $ \pi_{ik} $ is the membership indicator:  

## E-Step Update Rule

In every E step we calculate distance of each point in our dataset to all centroids and update $ \pi_{ik} $ as follows:

$$
\pi_{ik} =
\begin{cases}
1 & \text{if } k = \arg \min_{j} \| x_i - \mu_j \|^2 \\
0 & \text{otherwise}
\end{cases}
$$

## Proof that this approach reaches minimum

our objective here is to minimize the assignment cost to each of the centroid $ \mu_k $ for each data point $ x_i $:

$$
\min_{\pi_{ik}} \sum_{k=1}^{K} \pi_{ik} \| x_i - \mu_k \|^2
$$

As $ \pi_{ik} $ has only two values 0 or 1, if we assign a point to a cluster $ k $ were it's distance from point ${x_{i}}$ is minimum that is,

$$
\min_k \| x_i - \mu_k \|^2
$$

If we assign all the points such that the distance to the centroid it is assigned to is minimum our overall objective funtion reaches to it's minimum, which will not be the case when even a single point is assigned to any other cluster from which it's distance is not minimum.

Thus, the assignment:

$$
\pi_{ik} = 1 \quad \text{if } k = \arg \min_{j} \| x_i - \mu_j \|^2
$$

achieves global minimum for the current cluster centroids $ \mu_k $.


### Question 2

The K-Means clustering objective is to minimize the within-cluster sum of squared distances:

$$
J(\pi, \mu) = \sum_{i=1}^{N} \sum_{k=1}^{K} \pi_{ik} \| x_i - \mu_k \|^2
$$

where:

- $ x_i $  is the $ i $-th data point,
- $ \mu_k $ is the centroid of cluster $ k $,
- $ \pi_{ik} $ is the membership indicator:

  $$
  \pi_{ik} =
  \begin{cases}
  1 & \text{if } x_i \text{ is assigned to cluster } k \\
  0 & \text{otherwise}
  \end{cases}
  $$

## M-Step Update Rule

In the M-step, given the current cluster assignments $ \pi_{ik} $, the goal is to find the optimal centroids $ \mu_k $ that minimize the objective function:

$$
\min_{\mu_k} \sum_{i=1}^{N} \pi_{ik} \| x_i - \mu_k \|^2
$$

Taking the derivative of the objective function with respect to $ \mu_k $:

$$
\frac{\partial}{\partial \mu_k} \sum_{i=1}^{N} \pi_{ik} \| x_i - \mu_k \|^2 = \sum_{i=1}^{N} \pi_{ik} 2 ( \mu_k - x_i )
$$

To minimize we need to set the partial derivative of objective function to zero:

$$
\sum_{i=1}^{N} \pi_{ik} ( \mu_k - x_i ) = 0
$$

Rearranging this equation we get:

$$
\mu_k \sum_{i=1}^{N} \pi_{ik} = \sum_{i=1}^{N} \pi_{ik} x_i
$$

We get:

$$
\mu_k = \frac{\sum_{i=1}^{N} \pi_{ik} x_i}{\sum_{i=1}^{N} \pi_{ik}}
$$

From this equation we can say that the new $\mu_k$ is the average of all the points assigned to it. Hence, after some iteration we will see no update which we can consider as a stopping criteria or can say we the objective function has reached the global minimum.


### Question 3

In K-Means algorithm it is never the case where the E and M Step generate $\mu_k$ or $\pi_{ik}$ such that it increase the value of objective function. That means it can only decrease the objective function or there is no change in it's value which is the case when we can say that the algorithm has converged.<br>

The main thing in K-Means algorithm is that it depends on the initial selection of centroids. If centroids are not selected properly the algorithm will converge to it's local minima.<br>

For, example let's consider MNIST dataset it contains digits if we do random initialization of centroids at first than it may be possible that multiple centroids carry the same number. Which will result into a biased case where it will perform well for one class but not other. So, here if we select one centroid for each digit than the algorithm reaches it global maxima and performs very well.<br>

Hence, we can say that the K-Means algorithm is always going to converge no matter what but we cann't say with confidence that it has converged to local or global minima.

In [5]:
# For mnist dataset
def zero_mean_unit_variance_normalization(data):

    mnist_data_normalized_with_unit_var = []
    for image in data:
        image = np.reshape(image,(784,))
        pixel_value_mean = image.mean()
        pixel_value_std = image.std()
        # Shifting scale to [0,1]
        image = (image-pixel_value_mean)/(pixel_value_std)
        mnist_data_normalized_with_unit_var.append(image)
    return mnist_data_normalized_with_unit_var

mnist_train_images_normalized_with_unit_var = zero_mean_unit_variance_normalization(mnist_train_images)
mnist_test_images_normalized_with_unit_var = zero_mean_unit_variance_normalization(mnist_test_images)
fashion_mnist_train_images_normalized_with_unit_var = zero_mean_unit_variance_normalization(fashion_mnist_train_images)
fashion_mnist_test_images_normalized_with_unit_var = zero_mean_unit_variance_normalization(fashion_mnist_test_images)

In [6]:
np.array(mnist_train_images_normalized_with_unit_var).shape

(60000, 784)

# Problem 2

### Hard KMeans 

In [7]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity

import math
import time

class KMeans:
    def __init__(self, n_clusters, batch_size=10,metric='euclidean',max_iter=100,random_state=42):

        self.n_clusters = n_clusters
        self.batch_size = batch_size 
        self.random_state = random_state
        self.metric = metric
        self.max_iter = max_iter
        self.centroids = None 
        self.pi_matrix = None 
        self.labels_ = None  
        self.X = None 
        self.dist_matrix = None 
        self.no_of_batches = None  
    

    def update_pi_matrix(self, X, no_of_batches, batch_size):

        pi_matrix = np.zeros((X.shape[0], self.n_clusters))  
        dist_matrix = np.zeros((X.shape[0], self.n_clusters)) 
        
        for i in range(no_of_batches):
            # Calculate start and end indices for the current batch
            start = i * batch_size
            end = min(start + batch_size, X.shape[0])

            # Compute distances between centroids and data points in the current batch
            

            # Find the index of the closest centroid for each data point in the batch
            if self.metric=='cosine':
                # start_time = time.time()
                similarity = cosine_similarity(X=self.centroids, Y=X[start:end])
                dist_matrix[start:end, :] = similarity.T
                index_of_closest_centroids = np.argmax(similarity, axis=0)
                # end_time = time.time()
                
            else:
                distances = pairwise_distances(X=self.centroids, Y=X[start:end], metric=self.metric)
                index_of_closest_centroids = np.argmin(distances, axis=0)
                dist_matrix[start:end, :] = distances.T

            # Update the assignment matrix (pi_matrix) for the current batch
            pi_matrix[np.arange(start, end), index_of_closest_centroids] = 1

            # Update the distance matrix for the current batch
            

        return pi_matrix, dist_matrix

    def update_centroids(self, X, pi_matrix):
        
        # Compute new centroids as the mean of data points assigned to each cluster
        centroids = pi_matrix.T @ X
        no_of_members = np.sum(pi_matrix, axis=0)[:, np.newaxis]  # Number of members in each cluster
        centroids = np.divide(centroids, no_of_members, where=no_of_members != 0) 
        return centroids

    def get_labels(self, pi_matrix):

        return np.argmax(pi_matrix, axis=1)

    def fit(self, X):

        i = 0
        self.X = X
        np.random.seed(self.random_state)  # Set random seed for reproducibility
        self.no_of_batches = math.ceil(X.shape[0] / self.batch_size)  # Calculate number of batches

        # Initialize centroids by randomly selecting data points
        self.centroids = X[np.random.choice(X.shape[0], self.n_clusters, replace=False)]

        # Iterate until convergence or maximum iterations are reached
        while i <= self.max_iter:
            # Update assignment matrix (pi_matrix) and distance matrix
            # start_time = time.time()
            self.pi_matrix, self.dist_matrix = self.update_pi_matrix(X, self.no_of_batches, self.batch_size)
            # end_time = time.time()
            # exe_time = end_time-start_time
            # print('Time for this iter:%.2f' % exe_time)

            # Update centroids based on the current assignments
            new_centroids = self.update_centroids(X, self.pi_matrix)

            # Check for convergence (if centroids do not change)
            if np.array_equal(self.centroids, new_centroids):
                break

            # Update centroids for the next iteration
            self.centroids = new_centroids
            i += 1

        print(f'It took {i} iterations to converge')
        self.labels_ = self.get_labels(self.pi_matrix)  # Assign final cluster labels

    def predict(self, X):

        no_of_batches = math.ceil(X.shape[0] / 100)  # Use a fixed batch size of 100 for prediction
        self.pi_matrix, self.dist_matrix = self.update_pi_matrix(X, no_of_batches, 100)
        prediction = self.get_labels(self.pi_matrix)
        return prediction

    def gini_index(self, cf_matrix, by='row'):

        gini_index_per_cluster = []
        avg_gini_index = 0

        for i in range(self.n_clusters):
            if by == 'row':
                no_of_members = np.sum(cf_matrix[i, :])  # Number of members in the cluster
                prob = cf_matrix[i, :] / no_of_members if no_of_members > 0 else np.zeros(cf_matrix.shape[1])
            else:
                no_of_members = np.sum(cf_matrix[:, i])  # Number of members in the cluster
                prob = cf_matrix[:, i] / no_of_members if no_of_members > 0 else np.zeros(cf_matrix.shape[1])

            # Compute Gini index for the current cluster
            gini_index_of_cluster = 1 - np.sum(prob ** 2)
            gini_index_per_cluster.append(gini_index_of_cluster)
            avg_gini_index += (gini_index_of_cluster * no_of_members)

        # Compute weighted average Gini index
        avg_gini_index = avg_gini_index / np.sum(cf_matrix)

        return gini_index_per_cluster, avg_gini_index

    def evaluate(self, y_true, y_pred):

        cf_matrix = confusion_matrix(y_true, y_pred)  # Compute confusion matrix
        gini_per_cluster, avg_gini = self.gini_index(cf_matrix)  # Compute Gini index
        purity = self.purity_of_cluster(cf_matrix)  # Compute purity
        sum_of_squares = self.calculating_sse()  # Compute sum of squared errors (SSE)

        # Print evaluation metrics
        print('Confusion Matrix:\n', cf_matrix)
        print('\n Gini Index Per Cluster:\n ', gini_per_cluster)
        print('\n Average Gini Index: %.2f' % avg_gini)
        print('\n Purity of the Cluster: %.2f' % purity)
        print('\n Sum Of Squares: %.2f' % sum_of_squares)

    def purity_of_cluster(self, cf_matrix):

        P_i = np.sum(np.max(cf_matrix, axis=1))  # Sum of the maximum values in each row
        purity = P_i / np.sum(cf_matrix)  # Compute purity
        return purity

    def calculating_sse(self):

        return np.sum(self.pi_matrix * (self.dist_matrix ** 2))  # Compute SSE

### Soft KMeans

In [8]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import confusion_matrix
import math

class Soft_KMeans:
    def __init__(self, n_clusters,beta=0.1,metric='euclidean',max_iter=100 ,batch_size=10, random_state=42):

        self.beta = beta
        self.n_clusters = n_clusters
        self.metric=metric
        self.batch_size=batch_size
        self.max_iter=max_iter
        self.random_state=random_state
        self.X = None 
        self.centroids = None 
        self.pi_matrix = None 
        self.labels_ = None  
        self.dist_matrix = None 
        self.no_of_batches = None  
        
        
    def softmax_for_kmeans(self,distances):
        exp_distances = np.exp(-self.beta*distances)
        softmax_prob = exp_distances/np.sum(exp_distances,axis=0)
        return softmax_prob
        

    def update_pi_matrix(self, X, no_of_batches, batch_size):


        pi_matrix = np.zeros((X.shape[0], self.n_clusters))  
        dist_matrix = np.zeros((X.shape[0], self.n_clusters)) 

        for i in range(no_of_batches):
            # Calculate start and end indices for the current batch
            start = i * batch_size
            end = min(start + batch_size, X.shape[0])
            
            # Compute distances between centroids and data points in the current batch
            distances = pairwise_distances(X=self.centroids, Y=X[start:end], metric=self.metric)
            
            # Update the distance matrix for the current batch
            dist_matrix[start:end, :] = distances.T

            # Update the assignment matrix (pi_matrix) for the current batch
            pi_matrix[np.arange(start, end), :] = self.softmax_for_kmeans(distances.T)

        return pi_matrix, dist_matrix

    def update_centroids(self, X, pi_matrix):

        # Compute new centroids as the mean of data points assigned to each cluster
        centroids = pi_matrix.T @ X
        sum_of_prob = np.sum(pi_matrix, axis=0)[:, np.newaxis]  # Number of members in each cluster
        centroids = np.divide(centroids, sum_of_prob, where=sum_of_prob != 0) 
        return centroids
    
    def get_labels(self, pi_matrix):

        return np.argmax(pi_matrix, axis=1)


    def fit(self, X):

        i = 0
        self.X = X
        np.random.seed(self.random_state)  # Set random seed for reproducibility
        self.no_of_batches = math.ceil(X.shape[0] / self.batch_size)  # Calculate number of batches

        # Initialize centroids by randomly selecting data points
        self.centroids = X[np.random.choice(X.shape[0], self.n_clusters, replace=False)]

        # Iterate until convergence or maximum iterations are reached
        while i <= self.max_iter:
            # Update assignment matrix (pi_matrix) and distance matrix
            self.pi_matrix, self.dist_matrix = self.update_pi_matrix(X, self.no_of_batches, self.batch_size)

            # Update centroids based on the current assignments
            new_centroids = self.update_centroids(X, self.pi_matrix)

            # Check for convergence (if centroids do not change)
            if np.array_equal(self.centroids, new_centroids):
                break

            # Update centroids for the next iteration
            self.centroids = new_centroids
            i += 1

        print(f'It took {i} iterations to converge')
        self.labels_ = self.get_labels(self.pi_matrix)  # Assign final cluster labels

    def predict(self, X):

        no_of_batches = math.ceil(X.shape[0] / 100)  # Use a fixed batch size of 100 for prediction
        self.pi_matrix, self.dist_matrix = self.update_pi_matrix(X, no_of_batches, 100)
        prediction = self.get_labels(self.pi_matrix)
        return prediction

    def gini_index(self, cf_matrix, by='row'):

        gini_index_per_cluster = []
        avg_gini_index = 0

        for i in range(self.n_clusters):
            if by == 'row':
                no_of_members = np.sum(cf_matrix[i, :])  # Number of members in the cluster
                prob = cf_matrix[i, :] / no_of_members if no_of_members > 0 else np.zeros(cf_matrix.shape[1])
            else:
                no_of_members = np.sum(cf_matrix[:, i])  # Number of members in the cluster
                prob = cf_matrix[:, i] / no_of_members if no_of_members > 0 else np.zeros(cf_matrix.shape[1])

            # Compute Gini index for the current cluster
            gini_index_of_cluster = 1 - np.sum(prob ** 2)
            gini_index_per_cluster.append(gini_index_of_cluster)
            avg_gini_index += (gini_index_of_cluster * no_of_members)

        # Compute weighted average Gini index
        avg_gini_index = avg_gini_index / np.sum(cf_matrix)

        return gini_index_per_cluster, avg_gini_index

    def evaluate(self, y_true, y_pred):

        cf_matrix = confusion_matrix(y_true, y_pred)  # Compute confusion matrix
        gini_per_cluster, avg_gini = self.gini_index(cf_matrix)  # Compute Gini index
        purity = self.purity_of_cluster(cf_matrix)  # Compute purity
        sum_of_squares = self.calculating_sse()  # Compute sum of squared errors (SSE)

        # Print evaluation metrics
        print('Confusion Matrix:\n', cf_matrix)
        print('\n Gini Index Per Cluster:\n ', gini_per_cluster)
        print('\n Average Gini Index: %.2f' % avg_gini)
        print('\n Purity of the Cluster: %.2f' % purity)
        print('\n Sum Of Squares: %.2f' % sum_of_squares)

    def purity_of_cluster(self, cf_matrix):

        P_i = np.sum(np.max(cf_matrix, axis=1))  # Sum of the maximum values in each row
        purity = P_i / np.sum(cf_matrix)  # Compute purity
        return purity

    def calculating_sse(self):

        return np.sum(self.pi_matrix * (self.dist_matrix ** 2))  # Compute SSE

#### K-Means For MNIST Dataset

In [9]:
from sklearn.model_selection import train_test_split
X = np.array(mnist_train_images_normalized_with_unit_var)
y = np.array(mnist_train_labels)
X = X.reshape(X.shape[0],-1,)
y = y.reshape(y.shape[0],-1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

##### For K=10

In [10]:
kmeans_mnist = KMeans(10,5000)

In [11]:
kmeans_mnist.fit(X_train)

It took 47 iterations to converge


In [12]:
y_pred = kmeans_mnist.labels_
kmeans_mnist.evaluate(y_train,y_pred)

Confusion Matrix:
 [[  12  533    1  101  179   22   12 3871    0    8]
 [  15   46 2486   28    9    1   47    0 2764    7]
 [  29  115   73  304  185  120 3745   66   86   43]
 [ 114  808   10 3526   48   68  176   34   51   20]
 [1177  117   53    2  135 1914   24    8   32 1182]
 [ 134 2023   26 1673   98  186   45   94    7   54]
 [   0  334   29   29 3919  121  200   89   33    0]
 [1861   21   99    5    8  601   50   43   91 2239]
 [ 179 2534   49 1488   41  128   54   19   75  152]
 [1919   57   18   93    6 1324    9   31   36 1269]]

 Gini Index Per Cluster:
  [0.31820614021983407, 0.526405751577405, 0.37489700527709846, 0.4425063398812735, 0.6994334237689004, 0.6300560640489287, 0.3126064024378913, 0.6481139797883458, 0.6084319784834146, 0.6886446689377846]

 Average Gini Index: 0.52

 Purity of the Cluster: 0.59

 Sum Of Squares: 19516787.88


In [13]:
predict = kmeans_mnist.predict(X_test)

In [14]:
kmeans_mnist.evaluate(y_test,predict)

Confusion Matrix:
 [[  0 140   0  32  38   2   3 969   0   0]
 [  3   9 609   7   2   1  12   0 694   2]
 [  7  29  18  70  40  14 965  13  23  13]
 [ 22 216   1 934  17  20  42   4  18   2]
 [334  22  15   2  27 463  10   3   6 316]
 [ 37 508   6 406  25  45  15  21   4  14]
 [  0  79   4   8 975  23  46  22   6   1]
 [488   2  25   2   1 129  15  13  20 552]
 [ 43 611  21 350  11  37   8   4  18  29]
 [470  11   5  34   2 326   2  11   8 318]]

 Gini Index Per Cluster:
  [0.31445027163988315, 0.5243465830340546, 0.3384292937255077, 0.43354158272815735, 0.7022249101869839, 0.6338804735054482, 0.2913758694394256, 0.6392837597724514, 0.6091488843661428, 0.6949578024782661]

 Average Gini Index: 0.52

 Purity of the Cluster: 0.60

 Sum Of Squares: 4873648.89


##### For K=5

In [15]:
kmeans_mnist = KMeans(5,5000)
kmeans_mnist.fit(X_train)
predict = kmeans_mnist.predict(X_test)
kmeans_mnist.evaluate(y_test,predict)

It took 101 iterations to converge
Confusion Matrix:
 [[   2   38    0  938  206    0    0    0    0    0]
 [  16   24 1291    1    7    0    0    0    0    0]
 [  12   41  134  123  882    0    0    0    0    0]
 [  74  106   66  989   41    0    0    0    0    0]
 [ 705  377   19    1   96    0    0    0    0    0]
 [  72  359   26  566   58    0    0    0    0    0]
 [   0    3   34   28 1099    0    0    0    0    0]
 [ 569  614   50    8    6    0    0    0    0    0]
 [ 103  474  148  364   43    0    0    0    0    0]
 [ 693  421   16   32   25    0    0    0    0    0]]

 Gini Index Per Cluster:
  [0.3410678414901388, 0.06991830649537834, 0.42792976667717675, 0.3852814437751201, 0.5479862096259487]

 Average Gini Index: 0.18

 Purity of the Cluster: 0.69

 Sum Of Squares: 5494638.71


In [16]:
print("Value of Objective Function for K=5 is:%.2f"% kmeans_mnist.calculating_sse())

Value of Objective Function for K=5 is:5494638.71


##### For k=20

In [17]:
kmeans_mnist = KMeans(20,5000)
kmeans_mnist.fit(X_train)
predict = kmeans_mnist.predict(X_test)
kmeans_mnist.evaluate(y_test,predict)

It took 101 iterations to converge
Confusion Matrix:
 [[  0  61   1   5  32  14   8 482  21   0   0   2  11   0   0   0   0 529
    2  16]
 [  2   2   8   6   5   0   1   0   0   0 658  67   0 580   0   0   2   0
    0   8]
 [  0   3 440   7   3   1   8   0   4  10   6  20 181   6   4   0   2   9
  480   8]
 [  0  19  17 298   3   2 512   0 279   2  11  31   5   0   2   5   7   3
   16  64]
 [465  12   0   2  20   6   0   0   0  38   4  17  31  13   0 328 252   2
    7   1]
 [ 11 325   1 113  16   3 142   5 210   5   3   5  10   2   1  21   7  13
    1 187]
 [  1  34   1   4 566 453   0  10   6   0   1   0  62   4   0   1   0  16
    2   3]
 [ 31   4   2   2   0   0   0   8   0 433  19  17  19  15 528  80  80   4
    4   1]
 [ 13  25   6 177   6   1  28   5  64   8   5 323  10   9   2   8  14   2
    7 419]
 [337   5   0  19   0   0   5   2  10 150  12   8  16   3  27 230 345   8
    0  10]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0  

In [18]:
print("Value of Objective Function for K=20 is:%.2f"% kmeans_mnist.calculating_sse())

Value of Objective Function for K=20 is:4394167.90


#### Soft Kmeans for MNIST Dataset

In [19]:
kmeans_mnist = Soft_KMeans(n_clusters=10,beta=1,batch_size=5000)

In [20]:
kmeans_mnist.fit(X_train)

It took 101 iterations to converge


In [21]:
predict = kmeans_mnist.predict(X_train)
kmeans_mnist.evaluate(y_train,predict)

Confusion Matrix:
 [[  29  241  156   50  142  819  483 2727   77   15]
 [  32   64  310   15   32 3087   26    0 1836    1]
 [   4  129 1424   33  110 1977   89   96  896    8]
 [  23   38  328 1641   22  655 1618   88  410   32]
 [ 294  120  138  109  276 2976  129   97   21  484]
 [ 141  167  215  564   85 1597 1011  309  163   88]
 [   0 1243   76    4 1378 1751   26  206   70    0]
 [ 853   21  413   37   37 2545  155  107   63  787]
 [ 182   94  954  392   95 1733  447   92  540  190]
 [ 936   62  124  161  128 1967  198  107   22 1057]]

 Gini Index Per Cluster:
  [0.6236259035206018, 0.5545550557338381, 0.7012440723554327, 0.7443157860126999, 0.5676091180417844, 0.7811781944827879, 0.7095757459696939, 0.6808145121563964, 0.791008345652049, 0.7365088061531504]

 Average Gini Index: 0.69

 Purity of the Cluster: 0.46

 Sum Of Squares: 1552060.74


In [22]:
# For Beta 0.1
kmeans_mnist = Soft_KMeans(n_clusters=10,beta=0.1,batch_size=5000)
kmeans_mnist.fit(X_train)
predict = kmeans_mnist.predict(X_train)
kmeans_mnist.evaluate(y_train,predict)

It took 101 iterations to converge
Confusion Matrix:
 [[1497  500  261   91   90  100  154   91 1369  586]
 [1676  464  225  183  114   82  234  109 1698  618]
 [1422  407  214  162  134   65  251   89 1464  558]
 [1412  500  210  167  120   80  281  100 1486  499]
 [1437  399  135  204  106   46  236   82 1471  528]
 [1317  417  207  116  122   61  222   79 1283  516]
 [1401  402  170  172   94   67  241   81 1551  575]
 [1624  434  176  136  130   70  241   95 1488  624]
 [1119  432  189  147  123   78  424   68 1615  524]
 [1288  417  173  174  134   78  367   70 1490  571]]

 Gini Index Per Cluster:
  [0.7847075006853869, 0.7787140143179002, 0.7883518537465249, 0.7928519867082573, 0.777323374292707, 0.7902292467455244, 0.7788952252240613, 0.7788896600377342, 0.7941291917481017, 0.7970242863097379]

 Average Gini Index: 0.79

 Purity of the Cluster: 0.32

 Sum Of Squares: 2560193.66


In [23]:
# For Beta 10
kmeans_mnist = Soft_KMeans(n_clusters=10,beta=10,batch_size=5000)
kmeans_mnist.fit(X_train)
predict = kmeans_mnist.predict(X_train)
kmeans_mnist.evaluate(y_train,predict)

It took 101 iterations to converge
Confusion Matrix:
 [[  14  287  124   57   60  545  404 2719  522    7]
 [ 149  142 1056   17   33 2300   80    0 1620    6]
 [  15  176 1403   13   65 1501  306  130 1153    4]
 [  70   40  442 1358   16  435 1314  168  998   14]
 [ 415  138  165   73  165 2882  105  211   85  405]
 [ 182  230  186  615   69 1118  690  405  739  106]
 [   4 1000   83    2 1138 1601   10  315  600    1]
 [1120   22  504   28   29 1914  129  395   97  780]
 [ 298  148  602  348   80 1265  969  144  657  208]
 [1201   55  111  130   70 1657  106  291  103 1038]]

 Gini Index Per Cluster:
  [0.6335167577020357, 0.68896999901584, 0.7491953688606647, 0.7884466627989236, 0.5927168580354487, 0.8433279534498502, 0.7644089188052167, 0.7631326745438224, 0.8367480638185141, 0.7614653208825786]

 Average Gini Index: 0.74

 Purity of the Cluster: 0.38

 Sum Of Squares: 1644676.56


#### KMeans for Fashion MNIST Dataset

In [24]:
X1 = np.array(fashion_mnist_train_images_normalized_with_unit_var)
y1 = np.array(fashion_mnist_train_labels)
X1 = X1.reshape(X1.shape[0],-1,)
y1 = y1.reshape(y1.shape[0],-1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

##### For k = 10

In [25]:
kmeans_fashion_mnist = KMeans(10,5000)

In [26]:
kmeans_fashion_mnist.fit(X_train)

It took 101 iterations to converge


In [27]:
y_pred = kmeans_fashion_mnist.labels_
kmeans_fashion_mnist.evaluate(y_train,y_pred)

Confusion Matrix:
 [[  19  404    1  101 3977    5   15  179   66    0]
 [   4   25 2427   20    0   14    4    8   75 2800]
 [  97  108   83  372   64   28   43  168 3741   86]
 [  83  998    8 3363   24   99   21   52  222   56]
 [1935   29   53    2   11 1125 1264  119   69   29]
 [ 194 1839   31 1469   99  115  176   84  342    8]
 [  94  236   32   28   91    0    1 3946  268   41]
 [ 560    9   99    7   45 2059 2081    6   59   78]
 [ 147 2628   67 1264   15  134  183   42  115   64]
 [1422   52   23   87   34 1826 1254    7   10   33]]

 Gini Index Per Cluster:
  [0.29472138820272886, 0.5248613133954957, 0.3809399366285886, 0.4898650246959919, 0.6914698041586236, 0.6967587396789698, 0.29948194605602385, 0.6442191051549363, 0.6037583315170387, 0.692058874254863]

 Average Gini Index: 0.53

 Purity of the Cluster: 0.59

 Sum Of Squares: 19527679.30


##### For k=5

In [28]:
kmeans_fashion_mnist = KMeans(5,5000)
kmeans_fashion_mnist.fit(X_train)
predict = kmeans_fashion_mnist.predict(X_test)
kmeans_fashion_mnist.evaluate(y_test,predict)

It took 43 iterations to converge
Confusion Matrix:
 [[   8   59    2  989   98    0    0    0    0    0]
 [   5   27 1323    0   10    0    0    0    0    0]
 [  30  156   99   21  862    0    0    0    0    0]
 [  36 1058   49   15   47    0    0    0    0    0]
 [ 946    7   43    8  202    0    0    0    0    0]
 [ 164  602  118   29  151    0    0    0    0    0]
 [   2   16   32   27 1104    0    0    0    0    0]
 [1116    6   98   28   14    0    0    0    0    0]
 [ 150  645  293   10   94    0    0    0    0    0]
 [1064   45   51   12   29    0    0    0    0    0]]

 Gini Index Per Cluster:
  [0.2582149998204045, 0.060133370902601735, 0.4293297874835803, 0.22487904822575366, 0.3552951109570994]

 Average Gini Index: 0.13

 Purity of the Cluster: 0.81

 Sum Of Squares: 5480021.32


In [29]:
print("Value of Objective Function for K=5 is: %.2f"% kmeans_fashion_mnist.calculating_sse())

Value of Objective Function for K=5 is: 5480021.32


##### For k=20

In [30]:
kmeans_fashion_mnist = KMeans(20,5000)
kmeans_fashion_mnist.fit(X_train)
predict = kmeans_fashion_mnist.predict(X_test)
kmeans_fashion_mnist.evaluate(y_test,predict)

It took 88 iterations to converge
Confusion Matrix:
 [[  0  10   0  13   1   0   1  20   8   0 469 537   1   1   0  51  15  27
    2   0]
 [  0   1 462   9   1   2   4   0   0 461   0   0   0   2   0   3   0   6
    4 410]
 [ 28  15   4  15   3  10  38  12   0   3   3   9 486 496   6   8  10  12
    3   7]
 [  5 477   4 223   0   6  62   3  13   5   3   5  22  14   0  15 332   1
   14   1]
 [384   0   3   3 444  34   3  12   0   3   0   1   6   0   1  38   0  21
  246   7]
 [ 24  82   2  49   4   3   4  13 356   3   9  13   1   0   0 248 232  14
    5   2]
 [  6   0   2   3   1   0   0 528   6   0  14  10   1   7   0  16   6 580
    0   1]
 [ 82   0  14   2  20 483   3   1   0  15   7   3   4   7 497  10   2   0
  100  12]
 [ 11  34   9 327  13  10 562   7  29   8   8   4   5   3   0  50  86   4
   14   8]
 [288   3   5  18 213 203   1   1   4   5   5   4   3   0  17   6   6   1
  417   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   

In [31]:
print("Value of Objective Function for K=20 is: %.2f"% kmeans_fashion_mnist.calculating_sse())

Value of Objective Function for K=20 is: 4349024.49


### KMeans for 20 News Group Data

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english',max_features=10000,token_pattern='[A-Za-z]+',use_idf=False)  # Remove stopwords and apply TF-IDF
ng_group_tf_vector = vectorizer.fit_transform(newsgroups_data.data)

In [33]:
X_train,X_test,y_train,y_test = train_test_split(ng_group_tf_vector,newsgroups_data.target,test_size=0.2)

In [34]:
print('Size of training data:',X_train.shape)

Size of training data: (15076, 10000)


In [35]:
print('Size of test data:',X_test.shape)

Size of test data: (3770, 10000)


#### For k=20

In [36]:
kmeans_20ng = KMeans(n_clusters=20,metric='cosine',max_iter=100,batch_size=100)

In [37]:
kmeans_20ng.fit(X_train)

It took 44 iterations to converge


In [38]:
predicted = kmeans_20ng.predict(X_train)
kmeans_20ng.evaluate(y_train,predicted)

Confusion Matrix:
 [[ 40   1 168   8  19  83  29  22 112   8   3  42   4  69   8  25   0   5
    4   3]
 [ 34  35  74  60  70  63  39   3   0  56   1  61  57   6  15  26  75  40
    4  62]
 [ 35  34  77  20  31  57  45   5   3  36   1  23  27   5   5  20 286  41
   15  28]
 [ 32  35  88  12  44  69  43   7   0  14   1  29  52   5  14  25  40  47
  160  50]
 [ 44  33 100  13  61  96  55   8   0  19   0  40  43   8  26  25  18  37
  113  18]
 [ 28  23  69  23  39  48  26   6   1  22   1  24  31   6   6  21  55  51
    0 300]
 [ 37  59  39  17  12  58  43   3   3  41  13  20  26   4 220  20  20  16
   82  55]
 [ 74  18 139  22  24 162  29  17   3  12   3  36  19  28 111  55   2  24
    9   6]
 [ 61  11 169  20  22 146  62  23   7  12   5  54  19  15  59  65   3  16
   14   3]
 [ 51  22 129  18  19 177  27  24   2  20 183  33  16  13  16  37   0   6
    1   0]
 [ 58  12 116  15  18 133  32  18   4   8 319  17   9  16   7  18   1   1
    0   0]
 [ 44  18 151  25  30 139  35  13   5  20   1 

#### For k=10

In [39]:
kmeans_20ng = KMeans(n_clusters=10,metric='cosine',max_iter=100,batch_size=100)
kmeans_20ng.fit(X_train)
predict = kmeans_20ng.predict(X_test)
kmeans_20ng.evaluate(y_test,predict)

It took 21 iterations to converge
Confusion Matrix:
 [[20  1 42 10  4 18  3  3 44  1  0  0  0  0  0  0  0  0  0  0]
 [17 24 22 10 21 21 20  2  3 52  0  0  0  0  0  0  0  0  0  0]
 [13 12 26 13 14 21 12  1  4 75  0  0  0  0  0  0  0  0  0  0]
 [16 17 26 10 17 31 20  2  1 75  0  0  0  0  0  0  0  0  0  0]
 [33  2 21 21 23 26 16  4  3 57  0  0  0  0  0  0  0  0  0  0]
 [12 83 13 17 15 20  7  2  2 37  0  0  0  0  0  0  0  0  0  0]
 [56 12 16 16  2 24  8  1  2 50  0  0  0  0  0  0  0  0  0  0]
 [58  3 41 21  9 32 10  7  3 13  0  0  0  0  0  0  0  0  0  0]
 [54  1 42 20  6 55 14  5  5  8  0  0  0  0  0  0  0  0  0  0]
 [49  0 50 12  8 58  4 10  5  4  0  0  0  0  0  0  0  0  0  0]
 [51  0 39 15  7 66  6  6  6  1  0  0  0  0  0  0  0  0  0  0]
 [21  5 71  6  8 53 12  2 20 10  0  0  0  0  0  0  0  0  0  0]
 [42  3 30 23 22 28 18  1  2 23  0  0  0  0  0  0  0  0  0  0]
 [38  3 45 16 21 45  8  6 14  9  0  0  0  0  0  0  0  0  0  0]
 [47  2 39 21  5 36 10  7 14 12  0  0  0  0  0  0  0  0  0  0]
 [

In [40]:
print("Value of Objective Function for K=20 is: %.2f"% kmeans_20ng.calculating_sse())

Value of Objective Function for K=20 is: 302.52


#### For K=25 

In [41]:
kmeans_20ng = KMeans(n_clusters=25,metric='cosine',max_iter=100,batch_size=100)
kmeans_20ng.fit(X_train)
predict = kmeans_20ng.predict(X_test)
kmeans_20ng.evaluate(y_test,predict)

It took 45 iterations to converge
Confusion Matrix:
 [[ 8  2  8  6  5 12  2  2 20  0  1  7  1  4  2  5  0  0  0  1 15  3 17  2
  23]
 [ 6  7  3 22 14 14 12  2  3  7  2  9 15  4  0  4 10  4  0 20  1  9 12  5
   7]
 [ 8  6  4  4 10  8  7  1  3  8  0  3  5  2  1  9 56  4  3  7  0 12 12  7
  11]
 [ 4  6  3  2  7 19  9  1  1 35  0  6  7  5  7  5  9  6 35 10  0  3 10 14
  11]
 [ 9  6 10  8 18 15  8  1  3 13  0  8 12  4  5  9  0 12 30  1  0  2 13 13
   6]
 [ 5 10  7  5  7 14  7  0  1  2  3  4  5  5  1 11 14  9  1 75  0  6  3  6
   7]
 [ 7 25  2  3  2 15  4  1  2  4  3  7  7  3 39  7  4  3 16  9  0 10  5  0
   9]
 [17  2 14  4  8 20  8  5  2  2  0  4  5  6 26 14  0  2  3  3  1  6 15  9
  21]
 [10  0 21  5  3 41 16  4  7  1  0  4  4 10 13 16  0  1  3  1  2  9 16  1
  22]
 [14  3  5  3  5 40  2  6  2  0 53  3  3  6  2  7  0  0  0  0  2  4 18  0
  22]
 [13  1  5  4  2 33  5  5  4  0 65  1  4  8  2  8  0  0  0  0  1  3 13  1
  19]
 [ 6  3  4  2  3 28  8  0 13  0  0  5  1  9  1  3  1 60  0  1  0  4

In [42]:
print("Value of Objective Function for K=20 is: %.2f"% kmeans_20ng.calculating_sse())

Value of Objective Function for K=20 is: 374.48
